In [1]:
# Importação de bibliotecas
import pandas as pd

from pendulum import now

### Tabela 1613 - Área destinada à colheita, área colhida, quantidade produzida, rendimento médio e valor da produção das lavouras permanentes 

In [2]:
dataset_name = 'sidra_tab1613_lavoura_permanente'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

Área destinada à colheita (ha)

#### Bronze

In [3]:
culturas = [2717, 2718, 2719, 2720, 2721, 2722, 2723, 2724, 2725, 2726, 2727, 2728, 2729, 2730, 2731, 2732, 2733, 2734, 2735, 2736, 2737,
            2738, 2739, 2740, 2741, 2742, 2743, 2744, 2745, 2746, 2747, 2748, 45981, 90001]

contador = 1

ultimo_ano = now().year - 1

In [4]:
for ano in range(2003, ultimo_ano):
    for cultura in culturas:
        if contador == 1:
            df_1613 = pd.read_json(f'https://apisidra.ibge.gov.br/values/t/1613/n6/all/v/2313/p/{ano}/c82/{cultura}')
            contador += 1
        else:
            df_1613 = pd.concat(
                [df_1613,
                 (pd.read_json(f'https://apisidra.ibge.gov.br/values/t/1613/n6/all/v/2313/p/{ano}/c82/{cultura}')
                  .iloc[1:])],
                  ignore_index=True
                  )
            contador += 1

df_1613.to_json(f'{raiz}/bronze_{dataset_name}.json')

print(f'Foram contabilizadas {contador} repetições de dados de áreas de colheita raspados.')

Foram contabilizadas 681 repetições de dados de áreas de colheita raspados.


#### Prata

| Código | Cultura                        |
|--------|--------------------------------|
| 2717   | Abacate                        |
| 2718   | Algodão arbóreo (em caroço)   |
| 2719   | Azeitona                       |
| 2720   | Banana (cacho)                 |
| 2721   | Borracha (látex coagulado)     |
| 2722   | Cacau (em amêndoa)             |
| 2723   | Café (em grão) Total           |
| 2724   | Caqui                          |
| 2725   | Castanha de caju               |
| 2726   | Chá-da-índia (folha verde)     |
| 2727   | Coco-da-baía*                  |
| 2728   | Dendê (cacho de coco)          |
| 2729   | Erva-mate (folha verde)        |
| 2730   | Figo                           |
| 2731   | Goiaba                         |
| 2732   | Guaraná (semente)              |
| 2733   | Laranja                        |
| 2734   | Limão                          |
| 2735   | Maçã                           |
| 2736   | Mamão                          |
| 2737   | Manga                          |
| 2738   | Maracujá                       |
| 2739   | Marmelo                        |
| 2740   | Noz (fruto seco)               |
| 2741   | Pera                           |
| 2742   | Pêssego                        |
| 2743   | Pimenta-do-reino               |
| 2744   | Sisal ou agave (fibra)         |
| 2745   | Tangerina                      |
| 2746   | Tungue (fruto seco)            |
| 2747   | Urucum (semente)               |
| 2748   | Uva                            |
| 45981  | Açaí                           |
| 90001  | Palmito                        |

In [5]:
df_1613 = (pd.read_json(f'{raiz}/bronze_{dataset_name}.json')
          .drop(0, axis = 0)
          .filter(['D1C', 'D3C', 'D4C', 'V'])
          .assign(**{
              'uf': lambda df: df['D1C'].str.slice(0,2).astype(int),
              'hectares': lambda df: pd.to_numeric(df['V'], errors='coerce'),
              'municipio': lambda df: df['D1C'].astype(int),
              'ano': lambda df: df['D3C'].astype(int),
              'cultura': lambda df: df['D4C'].astype(int)
              })
          .convert_dtypes()
          .drop(['D1C', 'D3C', 'D4C', 'V'], axis='columns')
          .dropna(subset='hectares')
          .set_index(['uf', 'municipio', 'ano', 'cultura'])
          )

df_1613.to_parquet(f'{raiz}/prata_{dataset_name}.parquet')

### Ouro

In [6]:
dataset_ouro = 'ouro_volume_agricola_lavouras'

In [7]:
df_1613_final = pd.read_parquet(f'{raiz}/prata_{dataset_name}.parquet')

df_1613_final.to_csv(f'{raiz}/{dataset_ouro}_permanentes.csv')

In [8]:
df_1613_consolidado_munic = (df_1613_final.reset_index()
                             .groupby(['uf', 'municipio', 'ano'])
                             .agg({'hectares':sum})
                             )

df_1613_consolidado_munic.to_csv(f'{raiz}/{dataset_ouro}_permanentes_munic.csv')

In [9]:
df_1613_consolidado_uf = (df_1613_final.reset_index()
                          .groupby(['uf', 'ano'])
                          .agg({'hectares':sum})
                          )

df_1613_consolidado_uf.to_csv(f'{raiz}/{dataset_ouro}_permanentes_uf.csv')